# Convolutional Layer

- toc: true
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png
- hide: true

## Introduction


## Let's Start

In [154]:
import jax 
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time 

from typing import Tuple, List, Any, Dict, Callable

### Convolutional Layers in Keras

The first function argument is the number of filters and the second argument is the shape of the kernel. 

In [155]:
layer = tf.keras.layers.Conv2D(2, (2, 2), use_bias=False)

Arrays fed to `layer` must be a 4D tensor where the first axis is the batch size, the second axis is the width, the third axis is the height, and thr fourth axis is 
the number of input channels.

In [90]:
input_image = np.random.randn(4,4,3)

Because Keras layers operate on batches of data, and not individual examples, a batch dimension must be addes to `input_image`.  To do this in numpy, take `input_image` and add a `None` in the axis you want to add:

In [140]:
input_batch = input_image[None,:,:,:]

In [141]:
feature_maps = layer(input_batch)

In [142]:
feature_maps.shape

TensorShape([1, 3, 3, 2])

In [143]:
kernels = layer.get_weights()[0]

In [144]:
print(f'w shape = {kernels.shape}')

w shape = (2, 2, 3, 2)


In [157]:
def conv2d(kernel, x):
    y = np.zeros((3,3))
    
    y[0,0] = np.sum(kernel * x[0:2,0:2])
    y[0,1] = np.sum(kernel * x[0:2,1:3])
    y[0,2] = np.sum(kernel * x[0:2,2:4])
    y[1,0] = np.sum(kernel * x[1:3,0:2])
    y[1,1] = np.sum(kernel * x[1:3,1:3])
    y[1,2] = np.sum(kernel * x[1:3,2:4])
    y[2,0] = np.sum(kernel * x[2:4,0:2])
    y[2,1] = np.sum(kernel * x[2:4,1:3])
    y[2,2] = np.sum(kernel * x[2:4,2:4])    
    
    return y

In [196]:
def convolve(input_image, kernels):
    
    output = np.zeros((1,3,3,2))
    
    input_0 = input_image[:,:,0]
    input_1 = input_image[:,:,1]
    input_2 = input_image[:,:,2]
    
    # Different filters for each input channel
    w0 = kernels[:,:, 0, 0] 
    w1 = kernels[:,:, 1, 0]
    w2 = kernels[:,:, 2, 0]
    
    output_00 = conv2d(w0, input_0) # Apply filter to channel 0
    output_10 = conv2d(w1, input_1) # Apply filter to channel 1
    output_20 = conv2d(w2, input_2) # Apply filter to channel 2
    
    output[0,:,:,0] = output_00 + output_10 + output_20
    
    # Different filters for each input channel
    w0 = kernels[:,:, 0, 1] 
    w1 = kernels[:,:, 1, 1]
    w2 = kernels[:,:, 2, 1]
    
    output_01 = conv2d(w0, input_0) # Apply filter to channel 0
    output_11 = conv2d(w1, input_1) # Apply filter to channel 1
    output_21 = conv2d(w2, input_2) # Apply filter to channel 2
    
    output[0,:,:,1] = output_01 + output_11 + output_21
    
    

    # Add the filtered outputs from each channel
    return output

In [197]:

y = convolve(input_image, kernels)

In [198]:
assert np.all(np.isclose(feature_maps, y))